In [1]:
from Enceladus.jobs import TrainingPipeline
from Enceladus.utils import GetConfiguration

pipeline, model, sweep = GetConfiguration().run('config.ini')
worker = TrainingPipeline(
    config=pipeline,
    model_config=model,
    sweep_config=sweep,
    no_sweep=True,
)
worker.run()

2022-11-02 18:44:58.821115: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 18:44:58.826214: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 18:44:58.826461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 18:44:58.827129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Reading train split.
Reading val split.
Reading test split.
Fitting...
Epoch 1/20


2022-11-02 18:45:06.726917: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600


6560/6562 [============================>.] - ETA: 0s - loss: 0.4027 - mae: 0.4917

...

INFO:tensorflow:Assets written to: /home/cam/Documents/Enceladus/notebooks/wandb/run-20221102_184500-3969aajz/files/model-best/assets


INFO:tensorflow:Assets written to: /home/cam/Documents/Enceladus/notebooks/wandb/run-20221102_184500-3969aajz/files/model-best/assets
wandb: Adding directory to artifact (/home/cam/Documents/Enceladus/notebooks/wandb/run-20221102_184500-3969aajz/files/model-best)... Done. 0.4s


6562/6562 [==============================] - 173s 26ms/step - loss: 0.1709 - mae: 0.2979 - val_loss: 0.1913 - val_mae: 0.3051 - lr: 0.0010


INFO:tensorflow:Assets written to: enceladus_model/assets


INFO:tensorflow:Assets written to: enceladus_model/assets


In [ ]:
import keras
import numpy as np
import tensorflow as tf
from Enceladus.utils import RecordsHandler

model = keras.models.load_model('enceladus_model')

In [ ]:
handler = RecordsHandler(data_dir='/home/cam/Documents/database_tools/data-2022-10-28/mimic3/')
dataset = handler.read_records(n_cores=10, AUTOTUNE=tf.data.AUTOTUNE)
test = dataset['test']

ppg = []
abp = []
for p, a in test.take(10):
    ppg.append(p)
    abp.append(a)
ppg = np.array(ppg)
abp = np.array(abp)

In [ ]:
model.predict(ppg).shape